In [1]:
import numpy as np
from matplotlib import figure, gridspec,colors

import scanner_interpretation as scani

import healpy as hp

from matplotlib import colors,cm
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy import visualization as vis

import dill



# bands = ["u_5_15", "u_15_25", "u_25_35", "u_35_45", "u_45_55", "u_55_65", "u_65_75", "u_75_90", "l_15_5", "l_25_15", "l_35_25", "l_45_35", "l_55_45", "l_65_55", "l_75_65", "l_90_75", "center"]
# versions = ["hess1","hess1u"]
# paths = [f"/lustre/fs22/group/hess/user/wybouwt/full_scanner_survey/{version}/{band}" for band in bands for version in versions]
# # mplets = scani.Multiplets(paths[0])
# # mplets.appendMultiplets(*[scani.Multiplets(path) for path in paths[1:]])
# mplet_list = [scani.Multiplets(path) for path in paths]


# unicorns = [9,18]
# for j in unicorns:
#     mplet_list[j].objectifyColumns()

    
# mplets = mplet_list[0]
# mplets.appendMultiplets(*mplet_list[1:])
with open("pkl_jugs/n4/mplets_with_tevcat.pkl","rb") as f:
    mplets = dill.load(f)
with open("pkl_jugs/n3/mplets_bare.pkl","rb") as f:
    mplets3 = dill.load(f)

nosource_mask = mplets.table["TEVCAT_DISTANCES_DEG"] >= 0.2
Nmin4_mask = mplets.table["Nmax"] >= 4
Nmin5_mask = mplets.table["Nmax"] >= 5
dt1sec_mask = mplets.table["dt"] <= 1e9
xgal_mask = np.abs(mplets.table["MEDIAN_GLAT"]) > 5
print(type(mplets))

<class '__main__.Multiplets'>


In [3]:
mplets3.table[-2:]

Nmax,OBS_ID,ID,RA,DEC,TIME,ENERGY,dt,da,MEDIAN_RA,MEDIAN_DEC,SkyCoord,MEDIAN_GLAT,MEDIAN_GLON
,,,,,,,,,,,"deg,deg",deg,deg
int64,uint32,object,object,object,object,object,float64,float64,float64,float64,SkyCoord,float64,float64
3,154742,[6996501725461 7000796692490 7000796692518],[2.8270974 2.870535 2.9035206],[-71.98489 -71.826355 -71.82283 ],['2019-10-21T20:03:55.678841406' '2019-10-21T20:03:58.086445624'\n '2019-10-21T20:03:58.280102784'],[0.568209 0.86274356 0.70602965],2601261568.0,0.1791789905006963,2.870534896850586,-71.82635498046875,"2.870534896850586,-71.82635498046875",-44.96015748363038,307.3168937825529
3,154768,[18476949308189 18476949308455 18481244275023],[52.726414 52.701324 52.851723],[-37.314495 -37.151165 -37.21456 ],['2019-10-21T23:10:06.894969041' '2019-10-21T23:10:07.767560298'\n '2019-10-21T23:10:09.612289483'],[0.9164381 0.61229116 0.73589253],2717320448.0,0.18815909572729253,52.72641372680664,-37.214561462402344,"52.72641372680664,-37.214561462402344",-55.06298461947555,239.88232343514923


In [ ]:
pks2155_mask = mplets.table["TEVCAT_SOURCE_NAME"] == "PKS 2155-304"
mkr421_mask = mplets.table["TEVCAT_SOURCE_NAME"] == "Markarian 421"
mplets.table[mkr421_mask * ~nosource_mask]
# g = vis.hist(mplets.table[mkr421_mask * ~nosource_mask]["Nmax"],histtype="step")
# plt.yscale("log")

Cornerplot of the resulting multiplets

In [ ]:
def get_leftover(redu) -> np.ndarray:

# pval_0_mask = redu["LAMBDA_RATIO_SIGNIFICANCE"]==0.
    bernP_nan_mask = np.isnan(redu["BERNOULLI_P"])
    expP_nan_mask = np.isnan(redu["EXP_CORRECTED_P"])
    bernP_finite_mask = np.isfinite(redu["BERNOULLI_P"])
    expP_finite_mask = np.isfinite(redu["EXP_CORRECTED_P"])
    bernP_0_mask = redu["BERNOULLI_P"] == 0
    expP_0_mask = redu["EXP_CORRECTED_P"] == 0

    sanebern = ~bernP_nan_mask * bernP_finite_mask * ~bernP_0_mask
    saneexp = ~expP_nan_mask * expP_finite_mask * ~expP_0_mask

    leftover = sanebern * saneexp
    return leftover

In [ ]:
# plotting mplet results
from corner import corner, hist2d
from matplotlib import figure

# fig: figure.Figure = corner(mplets[get_leftover(mplets)].to_pandas(),var_names=["dt","da","BERNOULLI_SIGMA"])#, labels=[r"$\mathrm{d}t$ [ns]", r"$\mathrm{d}a$  [deg]"])
# fig: figure.Figure = corner(mplets.to_pandas(),var_names=["dt","da"], labels=[r"$\mathrm{d}t$ [ns]", r"$\mathrm{d}a$  [deg]"])
fig = figure.Figure(figsize=(5,3))
ax = fig.add_subplot()
hist2d(mplets.table["dt"]/1e9,mplets.table["da"],ax=ax,new_fig=False,bins=20)
       
labels=[r"$\mathrm{d}t$ [s]", r"$\mathrm{d}a$  [deg]"]
ax.set_xlabel(labels[0])
ax.set_ylabel(labels[1])
ax.set_xlim(0,3)
ax.set_ylim(0,0.3)
for ax in fig.get_axes():
    for spine in ["right","top"]:
        ax.spines[spine].set_visible(True)
ax.text(0.3,0.09,"Countour lines shown:\n0.5, 1, 1.5 and 2$\sigma$",horizontalalignment="left",verticalalignment="top")
fig.savefig("figures/dt_da_2dhist.png",facecolor="white")
# fig.savefig("figures/combined/n3_corner_dt_da.png",facecolor="white",dpi=300)

In [ ]:
max(mplets.table["Nmax"])

In [ ]:
from scipy.spatial import KDTree

In [ ]:
t = KDTree([[1,2],[3,4]])

In [ ]:
with open("pkl_jugs/n4/mplets_with_tevcat.pkl", "rb") as f:
    mplets_sorted = dill.load(f)
mplets_sorted.table.sort("Nmax")
A = mplets_sorted.table["TEVCAT_SOURCE_TYPE","TEVCAT_SOURCE_NAME"][::-1][:100]
from pprint import pprint
pprint(np.unique(A).tolist())

TeVCat distance histogram (keep!)

In [ ]:
# current_mask = nosource_mask
from matplotlib.figure import Axes

vline_sourcebound = .2
plt.figure(figsize=(6,3))
hist = vis.hist(mplets.table["TEVCAT_DISTANCES_DEG"],bins="freedman",histtype="step",stacked=True)
ax: Axes = plt.gca()
# ax.set_title("Multiplet distance to nearest TeVCat source.",loc="left")
ax.set_xlabel("Distance [deg]")
ax.set_ylabel("Counts")
ylim = ax.get_ylim()
# ax.vlines(vline_sourcebound,0,ylim[1],color="red",ls="--",label=f"{vline_sourcebound} deg")
# ax.set_yscale("log")
ax.set_xlim(-.5,8)
ax.fill_between([0,vline_sourcebound],*ylim, alpha=0.3, color="m",label="Associated with TeVCat source.")
ax.text(.3, 1e3, "Associated with know TeVCat source.")
# ax.text(2, 2650 + 50, "Associated with know TeVCat source.")
ax.arrow(1.9,2675 + 50,-1.5,0, lw=0.5)

# ax.fill_between([5,20],ylim[0],ylim[1]/5,color="green",alpha=0.3)
ax.text(3.5,25e1,"Tail: PNT not towards TeVCat source.")
ax.set_ylim(*ylim)
# ax.legend()
# ax.set_xscale("log")
plt.savefig("figures/n4_sourcedist_hist_makeup.png",facecolor="white",dpi=300)

HEALPix

In [2]:
from tevcat import TeVCat, Source
tc = TeVCat()
sources = tc.getSources()

Unknown observatory name found in GRB 160821B:  None
Unknown "discoverer" ID found in GRB 160821B: None
Unknown observatory name found in SNR G004.8+6.2:  None
Unknown "discoverer" ID found in SNR G004.8+6.2: None
Unknown observatory name found in 3HWC J0621+382:  None
Unknown "discoverer" ID found in 3HWC J0621+382: None
Unknown observatory name found in 3HWC J0630+186:  None
Unknown "discoverer" ID found in 3HWC J0630+186: None
Unknown observatory name found in 3HWC J0631+107:  None
Unknown "discoverer" ID found in 3HWC J0631+107: None


In [3]:
# Assuming you have a list of sources called 'sources'
def get_source_galactic(desired_name:str,sources: list[Source]):
    selected_source = None

    for source in sources:
        if source.canonical_name == desired_name:
            selected_source: Source = source
            break

    if selected_source is None:
        raise ValueError(f"{desired_name} not contained in sources")
    return selected_source.galactic

# Assuming you have a list of sources called 'sources'
def get_source_galactic(desired_name:str,sources: list[Source]):
    selected_source = None

    for source in sources:
        if source.canonical_name == desired_name:
            selected_source: Source = source
            break

    if selected_source is None:
        raise ValueError(f"{desired_name} not contained in sources")
    return selected_source.galactic

names = ["Crab", "PKS 2155-304", "Markarian 421"]
coords = [get_source_galactic(name,sources) for name in names]

In [4]:
fig,ax = scani.main_aitov(mplets)

xoffsets,yoffsets = np.radians([-5,-5,-10]),np.radians([0,0,0])

for name,coord,dx,dy in zip(names,coords,xoffsets,yoffsets):
    x,y = -np.radians([scani.convert_angle(coord.l.value)]),np.radians([coord.b.value])
    ax.scatter(x,y,color="black",marker=".",s=10,zorder=2)
    ax.text(x-dx,y-dy,name,fontsize="x-small")



fig.savefig("figures/labelled_aitov.png",facecolor="white",dpi=300)

In [ ]:
fig = plt.figure(dpi=300)

Nmax_mask = mplets.table["Nmax"] >= 4
dt_mask = mplets.table["dt"] <= 1e9
current_mask = nosource_mask*Nmax_mask*dt_mask

cmap = cm.jet
cmap.set_bad("r")
NSIDE = 2**5
m = scani.cat2hpx(mplets.table[current_mask]["MEDIAN_RA"],mplets.table[current_mask]["MEDIAN_DEC"],nside=NSIDE)
print(np.sqrt(hp.nside2pixarea(NSIDE,degrees=True)))
mmin, mmax = min(m),max(m)

hp.mollview(m,cmap=cmap, hold=True,nest=False,min=mmin,max=mmax,notext=True,fig=1,cbar=False,title="Transient tool applied on all H.E.S.S. data.")
#create a colorbar axis
cax = fig.add_axes([0.125, 0.1, 0.775, 0.03])

#create a colorbar and normalise it
# cmap.set_under("white")
# axs = fig.get_axes()
# axs[0].images[0].set_norm(colors.SymLogNorm(100))
cb = mpl.colorbar.ColorbarBase(cax,cmap=cmap,norm=mpl.colors.Normalize(vmin=mmin,vmax=mmax),
                               orientation='horizontal')
cb.set_label("Counts")
cb.solids.set_edgecolor("face")

plt.show()
# plt.savefig("figures/combined/healpy.png",facecolor="white")

Nmax histogram

In [ ]:
totalmask = nosource_mask * xgal_mask
nos = nosource_mask * ~xgal_mask
xgal = ~nosource_mask * xgal_mask
sourcegal = ~nosource_mask * ~xgal_mask


remaining_Nmax = max(mplets.table[totalmask]["Nmax"])
print(f"Nmax left after mask: {remaining_Nmax}")
nmax_data = mplets.table["Nmax"]
plt.figure(figsize=(6,3))
n,bins,_ = plt.hist([nmax_data[totalmask],nmax_data[nos],nmax_data[sourcegal],nmax_data[xgal]],bins=range(4,22),histtype="bar",stacked=True,color=["c","m","y","lightgrey"],label=["extragalactic, sourceless","galactic, sourceless","galactic, source assoc.","extragalatic, source assoc."],align="left")
plt.hist(nmax_data,color="black",histtype="step",bins=bins,label="Total",align="left")
ax = plt.gca()
ax.set_xlabel("$N_m$")
ax.set_ylabel("Counts")
ax.set_yscale("log")
ax.legend()
ax.set_xticks(range(4,22,2))
# ax.set_title("Multiplicity $N_m$ of multiplets.",loc="left")
# plt.show()
plt.savefig("figures/n4_Nmax_hist.png",facecolor="white",dpi=300)

In [ ]:
mplets.table.colnames

In [ ]:
tab = mplets.table[sourcegal]["TEVCAT_SOURCE_NAME","TEVCAT_DISTANCES_DEG","TEVCAT_SOURCE_TYPE"]
tab.group_by("TEVCAT_SOURCE_NAME")
tab.pprint_all()

In [ ]:
crab = np.asarray([item in ["Crab","Crab Nebula"] for item in mplets.table["TEVCAT_SOURCE_NAME"]])
a = sum(sourcegal)
b = sum(xgal)
print(sum(crab))
c = a/(a+b)

avar, bvar = a,b
cvar = (1/(a+b)**2)*(avar + (avar + bvar)*(a/(a + b))**2)
print(c,np.sqrt(cvar))

In [ ]:
# current_mask = nosource_mask# * mplets.table["Nmax"] > 3
temp = mplets.table[current_mask]["Nmax","OBS_ID","dt","da","MEDIAN_GLAT","TEVCAT_SOURCE_NAME","TEVCAT_SOURCE_TYPE","TEVCAT_DISTANCES_DEG"]
temp[:]

In [ ]:
temp.sort["Nmax"]

In [ ]:
Nmax = 3
run_count_rate = 1/60
expected_photons_3sec = 3*run_count_rate

expected_photons_3sec

In [ ]:
from math import factorial
sum = 1

for i in range(Nmax):
    sum -= expected_photons_3sec**(i)*np.exp(-expected_photons_3sec)/(factorial(i))
sum
                                                                      
                                                                    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial

@np.vectorize
def pvalue(lamb):
    Nmax = 3
    sum = 1
    for i in range(Nmax):
        sum -= lamb**(i)*np.exp(-lamb)/(factorial(i))
    return sum

dt = np.geomspace(0.1,3,10)
lamb = (1/60)*dt
pval = pvalue(lamb)
plt.scatter(dt,pval)
plt.xscale("log")
plt.yscale("log")
    
